 # Word Embeddings : le modèle Word2Vec de la décennie 1900 et 1910

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

# Word Embeddings : le modèle Word2Vec de la décennie 1910

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [43]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [44]:
infile = f"../data/tmp_tp4/sents_1910_tp4.txt"
sentences = MySentences(infile)

sentences = [sentence for sentence in sentences]

### Détection des bigrams

In [45]:
bigram_phrases = Phrases(sentences)

In [46]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [47]:
len(bigram_phrases.vocab.keys())

464915

Prenons une clé au hasard :

In [48]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

de_1901


Le dictionnaire indique le score de cette coocurrence :

In [49]:
bigram_phrases.vocab[key_]

9

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [50]:
%time bigram_phrases[sentences[78]]

CPU times: user 60 µs, sys: 7 µs, total: 67 µs
Wall time: 72 µs


['liberes', 'de', '25', 'francs', '(', 'verts', ')']

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [51]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [52]:
%time bigram_phraser[sentences[78]]

CPU times: user 0 ns, sys: 29 µs, total: 29 µs
Wall time: 31.7 µs


['liberes', 'de', '25', 'francs', '(', 'verts', ')']

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [53]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [54]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [55]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [56]:
print(corpus[:100])

[['(', '31'], ['janvier'], ['1910', ')'], ['--'], ['100'], ['--'], ['ces', 'comptes', 'soldent', 'par', 'un', 'deficit', 'de', '.'], ['.'], ['.'], ['fr', '.'], ['alors', 'que', 'la', 'somme_prevue', 'au', 'bu', 'iget', 'de', 'la', 'ville'], ['n', "'", 'est', 'que', 'de'], ['difference', '.'], ['626', '.', '568', '90'], ['532', ',', '970', '37'], ['.'], ['fr', '.'], ['93', ',', '598', '53'], ['en_vue', 'de', 'permettre', 'de', 'liquider', 'le', 'montant', 'de', 'ce', 'deficit', ',', 'le'], ['college_vous_propose', ',', 'messieurs', ',', 'le', 'vote', 'd', "'", 'un_credit_supplementaire', 'defr', '.'], ['9', ';', '5', ',', '598', '-', 'o3', 'a', 'l', "'", 'art', '.'], ['131', 'des', 'depenses_ordinaires'], ['du', 'budget', 'de', '1901', ')', ':', '<<', 'subside_eventuel', 'a'], ['l', "'", 'administration'], ['des', 'hospices', 'et', 'secours', '>>.'], ['les', 'ressources_ordinaires', 'de', 'l', "'", 'exercice', 'couvriront', 'la', 'depense', '.'], ['la', 'section', 'des_finances', 'a', '

## Entraînement d'un modèle Word2Vec sur ce corpus

In [57]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 26.9 s, sys: 158 ms, total: 27 s
Wall time: 15.5 s


### Sauvegarder le modèle dans un fichier

In [58]:
outfile = f"../data/bulletins_1910_tp4.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [59]:
model = Word2Vec.load("../data/bulletins_1910_tp4.model")

### Imprimer le vecteur d'un terme

In [60]:
model.wv["ecole"]

array([-1.1409361 ,  0.39057782,  0.44864193, -0.91632617,  3.1528392 ,
       -2.4987197 , -0.660255  , -0.76340646,  0.3704395 , -0.2604994 ,
        0.5134656 , -1.3344119 , -1.1363195 , -0.01562662, -2.2504041 ,
        1.258942  , -0.1365764 , -1.6202219 , -3.1713548 ,  0.4084466 ,
        1.0156811 ,  0.4022262 ,  0.6510475 , -2.3123777 ,  1.859781  ,
       -1.3063929 , -3.2964182 ,  1.7346755 , -3.7652917 , -0.7917346 ,
        1.0229253 , -1.1007062 ], dtype=float32)

### Calculer la similarité entre deux termes

In [61]:
model.wv.similarity("ecole", "college")

0.17750676

In [62]:
model.wv.similarity("ecole", "travaux")

0.4094169

In [63]:
model.wv.similarity("ecole", "construction")

0.66008663

In [64]:
model.wv.similarity("ecole", "enfants")

0.6647371

In [65]:
model.wv.similarity("ecole", "economie")

0.5932039

In [66]:
model.wv.similarity("ecole", "bourgmestre")

0.15416613

### Chercher les mots les plus proches d'un terme donné

In [67]:
model.wv.most_similar("ecole", topn=10)

[('instruction', 0.78495854139328),
 ('association', 0.7664194107055664),
 ('enseignement', 0.7567496299743652),
 ('usage', 0.7551409006118774),
 ('extension', 0.7540187835693359),
 ('organisation', 0.7433465719223022),
 ('habitation', 0.7421678304672241),
 ('hopital', 0.7350879907608032),
 ('industrie', 0.7328788638114929),
 ('etablissement', 0.722536563873291)]

### Faire des recherches complexes à travers l'espace vectoriel

In [68]:
print(model.wv.most_similar(positive=['art', 'travaux'], negative=['bruxelles']))

[('primes', 0.71915602684021), ('ordinaires', 0.7064698934555054), ('incendie', 0.6935310959815979), ('ameublement', 0.6762669682502747), ('education', 0.6470165848731995), ('adultes', 0.6413487792015076), ('electricite', 0.6357102990150452), ('institutrices', 0.6354878544807434), ('immondices', 0.6331477761268616), ('entretien', 0.6314980983734131)]


# Word Embeddings : le modèle Word2Vec de la décennie 1900

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [69]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [72]:
infile = f"../data/tmp_tp4/sents_1900_tp4.txt"
sentences = MySentences(infile)

sentences = [sentence for sentence in sentences]

### Détection des bigrams

In [73]:
bigram_phrases = Phrases(sentences)

In [74]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [75]:
len(bigram_phrases.vocab.keys())

478855

Prenons une clé au hasard :

In [76]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

950


Le dictionnaire indique le score de cette coocurrence :

In [77]:
bigram_phrases.vocab[key_]

328

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [78]:
%time bigram_phrases[sentences[78]]

CPU times: user 82 µs, sys: 0 ns, total: 82 µs
Wall time: 97.3 µs


['>>']

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [79]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [80]:
%time bigram_phraser[sentences[78]]

CPU times: user 26 µs, sys: 1 µs, total: 27 µs
Wall time: 33.6 µs


['>>']

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [81]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [82]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [83]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [84]:
print(corpus[:100])

[['produit', 'du', 'droit', 'de', 'frequent'], ['publics'], ['142'], ['4869'], ['p', 'u', 'p', 'i', 't', 'r', 'e', 's', 'de', 'l', 'i', 'q', 'u', 'i', 'd', 'a', 't', 'i', 'o', 'n'], ['>>'], ['nombf'], ['des'], ['redevances'], ['nombre'], ['montant'], ['total'], ['de', 'porteurs'], ['de', 'depeches'], ['de'], ['delegues'], ['d', "'", 'agents'], ['de', 'change'], ['1868'], ['b', 'o', 'u', 'r', 's', 'e', 'at'], ['c', 'ai'], ['c', '"'], ['montant'], ['sg'], ['a', 's'], ['d', "'", 'entrees'], ['?', 'n', 'm', 'o', 'y', 'e', 'n', 'n', 'e'], ['(', 'par', 'jour', ')'], ['d', 'e', 'sfonds'], ['nombre'], ['de'], ['banquiers'], ['annees'], ['bourse'], ['142'], ['5', ',', '600'], ['>>'], ['428'], ['128'], ['6', ',', '400'], ['>>'], ['>'], ['*'], ['4870'], ['445'], ['445'], ['7', ',', '250'], ['>>'], ['*'], ['>>'], ['>>'], ['4874'], ['4', '66'], ['466'], ['8', ',', '300'], ['>>'], ['>>'], ['s'], ['>>'], ['>>'], ['202'], ['40', ',', '400'], ['>>'], ['>'], ['>'], ['>>'], ['>>'], ['245'], ['55', ',', '

## Entrainement d'un modèle Word2Vec sur ce corpus

In [85]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 28.6 s, sys: 244 ms, total: 28.8 s
Wall time: 15.8 s


### Sauvegarder le modèle dans un fichier

In [87]:
outfile = f"../data/bulletins_1900_tp4.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [88]:
model = Word2Vec.load("../data/bulletins_1900_tp4.model")

### Imprimer le vecteur d'un terme

In [90]:
model.wv["ecole"]

array([-0.34754843, -0.17173974,  0.27266422,  0.61391026,  2.7732704 ,
        2.8321402 , -2.3571956 ,  1.80242   ,  0.6317885 ,  1.6909484 ,
       -0.05845789, -0.29225597, -2.0383966 ,  0.96590143,  0.14322126,
        0.45163924, -0.23044597,  0.37161425,  0.28837913, -1.0029634 ,
        2.6814685 , -0.43584177,  2.321995  ,  1.301112  ,  2.892728  ,
       -2.2201202 , -2.4498668 ,  0.5216067 , -1.768015  , -0.8668347 ,
        1.7438655 , -3.4809396 ], dtype=float32)

### Calculer la similarité entre deux termes

In [91]:
model.wv.similarity("ecole", "college")

0.17949401

In [92]:
model.wv.similarity("ecole", "travaux")

0.36352074

In [93]:
model.wv.similarity("ecole", "construction")

0.66064876

In [94]:
model.wv.similarity("ecole", "enfants")

0.5728574

In [95]:
model.wv.similarity("ecole", "economie")

0.60765934

In [96]:
model.wv.similarity("ecole", "bourgmestre")

0.0902953

### Chercher les mots les plus proches d'un terme donné

In [97]:
model.wv.most_similar("ecole", topn=10)

[('usine', 0.8174176216125488),
 ('hopital', 0.8038303256034851),
 ('ecole_primaire', 0.778652012348175),
 ('enseignement', 0.7499747276306152),
 ('academie', 0.7465783357620239),
 ('installation', 0.7443757057189941),
 ('ecole_professionnelle', 0.7415958642959595),
 ('association', 0.738534152507782),
 ('hospice', 0.7264813184738159),
 ('achevement', 0.7249936461448669)]

### Faire des recherches complexes à travers l'espace vectoriel

In [98]:
print(model.wv.most_similar(positive=['art', 'travaux'], negative=['bruxelles']))

[('abonnes', 0.6559460759162903), ('ressources_ordinaires', 0.6498435735702515), ('amortissement', 0.6362881660461426), ('incendie', 0.6327128410339355), ('ordinaires', 0.6271665096282959), ('ressources_extraordinaires', 0.6244856119155884), ('utilite_publique', 0.6229644417762756), ('exercice_1907', 0.6220771074295044), ('orgue', 0.6115286350250244), ('exercice_1908', 0.6092736721038818)]
